# ZHOU CHENGYANG
# 3036167854

## Q1 - top movies

In [34]:
from urllib.request import *
import re
import pandas as pd

In [35]:
class Q1Movies:
    def __init__(self):
        self.movies = {}
        self.movie_fields = ["name", "year", "rating", "vote", "length", "vote2"]
        for field in self.movie_fields:
            self.movies[field] = []

    def add_movie(self, movie):
        year = int(movie[2])
        hour, minute, length = int(movie[5]) // 3600, (int(movie[5]) // 60) % 60, ""
        if hour > 0:
            length += str(hour) + "h "
        if minute > 0:
            length += str(minute) + "m"

        self.movies["name"].append(movie[1])
        self.movies["year"].append(int(movie[2]))
        self.movies["rating"].append(float(movie[3]))
        self.movies["vote"].append(movie[4])
        self.movies["length"].append(length)
        self.movies["vote2"].append(int(movie[4]))

    def add_movies(self, movies, votes=None):
        for movie in movies:
            self.add_movie(movie)
        if votes:
            for i, vote in enumerate(votes):
                self.movies['vote'][i] = vote
                if vote[-1] == 'M':
                    vote2 = round(float(vote[:-1]) * 1000000)
                elif vote[-1] == 'K':
                    vote2 = round(float(vote[:-1]) * 1000)
                else:
                    vote2 = int(vote)
                self.movies['vote2'][i] = vote2

In [36]:
# name: '>\d+\.+(.+?)</h3'
# year: '>\d{4}</span'
# length: '>\d+[hm].*?</span'
# grade: ''
# vote: '>([0-9.]+[MK])<'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/116.0.0.0 Safari/537.36'}
url = "https://www.imdb.com/chart/top"
html = urlopen(Request(url=url, headers=headers))
html_text = bytes.decode(html.read())

results = re.findall(
    '"currentRank":(\d+).*?"titleText":{"text":"(.*?)".*?"releaseYear":{"year":(\d+).*?"ratingsSummary":{'
    '"aggregateRating":(\d+\.?\d*),"voteCount":(\d+).*?"runtime":{"seconds":(\d+),',
    html_text)
votes = re.findall('<!-- -->(\d+.?\d*[MK])<!-- -->', html_text)
data = Q1Movies()
data.add_movies(results, votes)
q1df = pd.DataFrame(data.movies)
q1df.to_csv('Q1_movies.csv')

show_years = {}
for i, year in enumerate(data.movies["year"]):
    if year >= 2020:
        if year in show_years:
            show_years[year]["rating"] += data.movies["rating"][i]
            show_years[year]["vote"] += data.movies["vote2"][i]
            show_years[year]["cnt"] += 1
        else:
            show_years[year] = {"rating": data.movies["rating"][i], "vote": data.movies["vote2"][i], "cnt": 1}
years = show_years.keys()
ratings = [round(value["rating"] / value["cnt"], 1) for value in show_years.values()]
votes = [value["vote"] // value["cnt"] for value in show_years.values()]
q1sample = pd.DataFrame({"year": years, "Ave_rating": ratings, "Ave_vote": votes})
q1sample = q1sample.reindex([1, 3, 2, 0])
q1sample.reset_index(inplace=True,drop=True)
q1sample.set_index(['year'], inplace=True)

In [37]:
q1df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    250 non-null    object 
 1   year    250 non-null    int64  
 2   rating  250 non-null    float64
 3   vote    250 non-null    object 
 4   length  250 non-null    object 
 5   vote2   250 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.8+ KB


In [38]:
q1df.head(3)

,name,year,rating,vote,length,vote2
0,The Shawshank Redemption,1994,9.3,2.8M,2h 22m,2800000
1,The Godfather,1972,9.2,2M,2h 55m,2000000
2,The Dark Knight,2008,9.0,2.8M,2h 32m,2800000


In [39]:
q1df.tail(3)

,name,year,rating,vote,length,vote2
247,Life of Brian,1979,8.0,415K,1h 34m,415000
248,Aladdin,1992,8.0,449K,1h 30m,449000
249,Drishyam,2015,8.2,91K,2h 43m,91000


In [40]:
q1sample

,Ave_rating,Ave_vote
year,,
2020,8.2,142000
2021,8.5,522000
2022,8.3,632000
2023,8.6,358000


## Q2 - fake jobs

In [41]:
from urllib.request import *
import pandas as pd
from bs4 import BeautifulSoup

In [42]:
class Q2Jobs:
    def __init__(self):
        self.jobs = {}
        self.job_fields = ["position", "company", "city", "state"]
        for field in self.job_fields:
            self.jobs[field] = []

    def add_jobs(self, positions, companies, locations):
        self.jobs["position"] = positions
        self.jobs["company"] = companies
        for location in locations:
            l = location.split(', ')
            self.jobs["city"].append(l[0])
            self.jobs["state"].append(l[1])

In [43]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/116.0.0.0 Safari/537.36'}
url = "https://realpython.github.io/fake-jobs"
html = urlopen(Request(url=url, headers=headers))
html_text = bytes.decode(html.read())

soup = BeautifulSoup(html_text, 'html.parser')
positions = [tag.string for tag in soup.find_all("h2", "title is-5")]
companies = [tag.string for tag in soup.find_all("h3", "subtitle is-6 company")]
locations = [tag.string.replace('\n', '').strip() for tag in soup.find_all("p", "location")]

data = Q2Jobs()
data.add_jobs(positions, companies, locations)
q2df = pd.DataFrame(data.jobs)
q2df.to_csv('Q2_jobs.csv')

q2sample = q2df.query("state == 'AA' & (position.str.contains('engineer') | position.str.contains('Engineer'))")

In [44]:
q2df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   position  100 non-null    object
 1   company   100 non-null    object
 2   city      100 non-null    object
 3   state     100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [45]:
q2df.head(3)

,position,company,city,state
0,Senior Python Developer,"Payne, Roberts and Davis",Stewartbury,AA
1,Energy engineer,Vasquez-Davidson,Christopherville,AA
2,Legal executive,"Jackson, Chambers and Levy",Port Ericaburgh,AA


In [46]:
q2df.tail(3)

,position,company,city,state
97,Database administrator,Yates-Ferguson,Port Susan,AE
98,Furniture designer,Ortega-Lawrence,North Tiffany,AA
99,Ship broker,"Fuentes, Walls and Castro",Michelleville,AP


In [47]:
q2sample

,position,company,city,state
1,Energy engineer,Vasquez-Davidson,Christopherville,AA
28,Structural engineer,Pierce-Long,Herbertside,AA
32,Broadcast engineer,"Morgan, Butler and Bennett",Loribury,AA
48,"Engineer, broadcasting (operations)",Taylor PLC,Gileston,AA


## Q3 - JP mountains

In [48]:
from urllib.request import *
import re
import pandas as pd
from bs4 import BeautifulSoup
import lxml

In [49]:
class Q3Mountains:
    def __init__(self):
        self.mountains = {}
        self.mountain_fields = ["name", "region", "elev", "id", "lat", "long", "elev_cat"]

        for field in self.mountain_fields:
            self.mountains[field] = []

    def add_mountains(self, mountains):
        self.mountains["name"] = [mountain[1] for mountain in mountains]
        self.mountains["region"] = [mountain[2] for mountain in mountains]
        self.mountains["elev"] = [int(mountain[3]) for mountain in mountains]
        self.mountains["id"] = [mountain[0] for mountain in mountains]
        self.set_locations()
        self.set_elev_cat()

    def set_locations(self):
        for id in self.mountains['id']:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                              'Chrome/116.0.0.0 Safari/537.36'}
            url = "https://www.peakbagger.com/peak.aspx?pid=" + id
            html = urlopen(Request(url=url, headers=headers))
            html_text = bytes.decode(html.read())
            results = re.findall('Latitude/Longitude.*?<br/>(\d+\.*\d*), (\d+\.*\d*)', html_text)[0]
            self.mountains["lat"].append(float(results[0]))
            self.mountains["long"].append(float(results[1]))

    def set_elev_cat(self):
        for elev in self.mountains["elev"]:
            if elev < 1000:
                self.mountains["elev_cat"].append("Cat 1")
            elif elev < 2000:
                self.mountains["elev_cat"].append("Cat 2")
            elif elev < 3000:
                self.mountains["elev_cat"].append("Cat 3")
            else:
                self.mountains["elev_cat"].append("Cat 4")

In [50]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/116.0.0.0 Safari/537.36'}
url = "https://www.peakbagger.com/list.aspx?lid=5651"
html = urlopen(Request(url=url, headers=headers))
html_text = bytes.decode(html.read())

results = re.findall('<td><a href=peak\.aspx\?pid=(\d+)>(.*?)</a></td><td>(.*?)</td><td align=right>(\d+)</td>',
                        html_text)

data = Q3Mountains()
data.add_mountains(results)
q3df = pd.DataFrame(data.mountains)
q3df.to_csv('Q3_mountains.csv')

sample = {}
for i, region in enumerate(data.mountains["region"]):
    if region not in sample:
        sample[region] = {'Cat 1': 0, 'Cat 2': 0, 'Cat 3': 0, 'Cat 4': 0}
    sample[region][data.mountains["elev_cat"][i]] += 1
sample_data = {'elev_cat\nregion': sample.keys()}
for cat in ["Cat 1", "Cat 2", "Cat 3", "Cat 4"]:
    sample_data[cat] = [value[cat] for value in sample.values()]

q3sample = pd.DataFrame(sample_data)
q3sample = q3sample.reindex([1, 3, 0, 2, 4])
q3sample.reset_index(inplace=True,drop=True)

In [51]:
q3df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      100 non-null    object 
 1   region    100 non-null    object 
 2   elev      100 non-null    int64  
 3   id        100 non-null    object 
 4   lat       100 non-null    float64
 5   long      100 non-null    float64
 6   elev_cat  100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [52]:
q3df.head(3)

,name,region,elev,id,lat,long,elev_cat
0,Fuji-san,Kanto,3776,10882,35.360638,138.727347,Cat 4
1,Kita-dake,Chubu,3192,10866,35.674537,138.238833,Cat 4
2,Hotaka-dake,Chubu,3190,10840,36.289203,137.647986,Cat 4


In [53]:
q3df.tail(3)

,name,region,elev,id,lat,long,elev_cat
97,Ibuki-yama,Western Japan,1377,10879,35.417856,136.406373,Cat 2
98,Kaimon-dake,Western Japan,924,10937,31.179944,130.528397,Cat 1
99,Tsukuba-san,Kanto,877,10844,36.225403,140.106705,Cat 1


In [54]:
q3sample

,elev_cat\nregion,Cat 1,Cat 2,Cat 3,Cat 4
0,Chubu,0,4,29,12
1,Hokkaido,0,5,4,0
2,Kanto,1,3,8,1
3,Tohoku,0,11,9,0
4,Western Japan,1,12,0,0


## Q4 - MSc(BA) teachers

In [55]:
from urllib.request import *
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
import ssl
import requests
import os

In [56]:
class Q4Teachers:
    def __init__(self):
        self.teachers = {}
        self.teacher_fields = ["name", "university", "url"]

        for field in self.teacher_fields:
            self.teachers[field] = []

    def add_teachers(self, names, universities, urls):
        self.teachers["name"] = names
        self.teachers["university"] = universities
        self.teachers["url"] = urls

    def save_images(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        for i, name in enumerate(self.teachers["name"]):
            r = requests.get(self.teachers["url"][i], verify=False)
            with open("images/" + name + ".jpg", 'wb') as handle:
                handle.write(r.content)

In [57]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/116.0.0.0 Safari/537.36'}
url = "https://msc.hkubs.hku.hk/articles/13?op=10&cd=99"
html = urlopen(Request(url=url, headers=headers), context=ssl._create_unverified_context())
html_text = bytes.decode(html.read())

soup = BeautifulSoup(html_text, 'html.parser')
names = [tag.string.split(',')[0] for tag in soup.find_all("b", "name")]
universities = [', '.join(tag.string.split(', ')[1:-1]) for tag in soup.find_all("p", "desinfo")]
urls = ["https://msc.hkubs.hku.hk/" + item.get('src') for item in soup.select('div[class="col-md-4 col-sm-6 col-xs-12"] a img')]

data = Q4Teachers()
data.add_teachers(names, universities, urls)
data.save_images()

q4df = pd.DataFrame(data.teachers)
q4df.to_csv('Q4_teachers.csv')

In [58]:
q4df

,name,university,url
0,Prof. Haipeng SHEN,"The Wharton School of Business, University of ...",https://msc.hkubs.hku.hk//uploads/image/202208...
1,Dr. Hailiang CHEN,Purdue University,https://msc.hkubs.hku.hk//uploads/image/202205...
2,Prof. Xin WANG,Duke University,https://msc.hkubs.hku.hk//uploads/image/202205...
3,Prof. Zhenhui Jack JIANG,University of British Columbia,https://msc.hkubs.hku.hk//uploads/image/202205...
4,Dr. Wei ZHANG,Purdue University,https://msc.hkubs.hku.hk//uploads/image/202205...
5,Prof. Z. Max SHEN,Northwestern University,https://msc.hkubs.hku.hk//uploads/image/202205...
6,Dr. Zhanrui CAI,The Pennsylvania State University,https://msc.hkubs.hku.hk//uploads/image/202308...
7,Prof. Michael C. L. CHAU,Arizona University,https://msc.hkubs.hku.hk//uploads/image/202205...
8,Dr. Chao DING,University of Florida,https://msc.hkubs.hku.hk//uploads/image/202205...
9,Dr. Xing HU,"Stern School of Business, New York University",https://msc.hkubs.hku.hk//uploads/image/202205...


In [59]:
q4df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        25 non-null     object
 1   university  25 non-null     object
 2   url         25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes
